[주성분코드부여방법](https://www.health.kr/drug_info/basedrug/main_ingredient.html)

http://www.hira.or.kr/rf/medicine/getHistoryList.do?pgmid=HIRAA030035020000

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
url = "http://www.hira.or.kr/rf/medicine/getHistoryList.do?pgmid=HIRAA030035020000"
query = "isActivity=Y&pageIndex=1&isDown=N&radio1=&searchWrd=&sortOrdr=&srchCnd=&artcNm=&mnfCoNm=&anceDdFr=&anceDdTo=&adtStaDdFr=&adtStaDdTo="

In [3]:
query_split = query.split("&")
params = {}
for q in query_split:
    q_split = q.split("=")
    key = q_split[0]
    val = q_split[1]
    params[key] = val
    
params

{'isActivity': 'Y',
 'pageIndex': '1',
 'isDown': 'N',
 'radio1': '',
 'searchWrd': '',
 'sortOrdr': '',
 'srchCnd': '',
 'artcNm': '',
 'mnfCoNm': '',
 'anceDdFr': '',
 'anceDdTo': '',
 'adtStaDdFr': '',
 'adtStaDdTo': ''}

In [4]:
# 원하는 약품코드 입력
params["artcNm"] = "111001ATE"
params

{'isActivity': 'Y',
 'pageIndex': '1',
 'isDown': 'N',
 'radio1': '',
 'searchWrd': '',
 'sortOrdr': '',
 'srchCnd': '',
 'artcNm': '111001ATE',
 'mnfCoNm': '',
 'anceDdFr': '',
 'anceDdTo': '',
 'adtStaDdFr': '',
 'adtStaDdTo': ''}

In [5]:
def get_medicine_table(url, params):
    
    response = requests.post(url, params)
    html = bs(response.text)
    
    table = html.select("table.tbl_default")
    if table:
        return table[0]
    else:
        return False

In [6]:
# 원하는 약품코드 
params["artcNm"] = "111001ATE"
params["artcNm"] = ""


table = get_medicine_table(url, params)
table

<table class="tbl_default lLine" summary="약품이력정보의 제품코드, 제품명, 업체명, 규격, 단위, 급여구분, 투여경로, 약효분류번호, 저가필수구분, 전문일반구분, 생동인정제품, 주성분코드, 임의조제불가구분, 사용장려비용가산, 상한가, 예외약1, 예외약2, 적용개시일자, 적용종료일자 제공표">
<caption>약품이력정보</caption>
<colgroup>
<col style="width:12%"/>
<col style="width:*%"/>
<col style="width:6%"/>
<col style="width:9%"/>
<col style="width:10%"/>
<col style="width:10%"/>
<col style="width:13%"/>
<col style="width:10%"/>
<col style="width:10%"/>
<col style="width:12%"/>
</colgroup>
<thead>
<tr>
<th class="fst" rowspan="2" scope="col">
                                        제품코드
                                        <span class="th-sort"><a class="desc" href="#none" onclick="setSortOrder('02-01'); return false;" title="오름차순 정렬">오름차순</a><a class="asc" href="#none" onclick="setSortOrder('02-02'); return false;" title="내림차순 정렬">내림차순</a></span>
</th>
<th scope="col">
                                        제품명
                                        <span class="th-sort"><a class="desc" href="#n

In [7]:
summary = table["summary"]
summary = summary.split()
cols = summary[1:-1]
cols

['제품코드,',
 '제품명,',
 '업체명,',
 '규격,',
 '단위,',
 '급여구분,',
 '투여경로,',
 '약효분류번호,',
 '저가필수구분,',
 '전문일반구분,',
 '생동인정제품,',
 '주성분코드,',
 '임의조제불가구분,',
 '사용장려비용가산,',
 '상한가,',
 '예외약1,',
 '예외약2,',
 '적용개시일자,',
 '적용종료일자']

In [8]:
len(cols)

19

In [9]:
trs = table.select("tbody > tr")

In [10]:
trs[1].select("td")

[<td class="blno">일동제약(주)</td>,
 <td class="ac">병</td>,
 <td class="ac">주사</td>,
 <td class="ac"></td>,
 <td class="ac"></td>,
 <td class="ac"></td>,
 <td class="ac">6,294</td>,
 <td class="ac">-</td>,
 <td class="ac"></td>]

In [11]:
def parse_tr(trs):
    row_list = []
    drug = []
    for i, tr in enumerate(trs):
        rows = tr.get_text().split("\n")
        drug.extend(rows)
        # 2개의 행에 데이터가 나눠 있기 때문에
        # 짝수행에서는 append 로 리스트에 추가하고 drug을 초기화 합니다.
        if i % 2 == 1:
            row_list.append(drug)
            drug = []
    return row_list

In [12]:
drug = parse_tr(trs)
df = pd.DataFrame(drug)
df.shape

(10, 23)

In [13]:
cols = ['제품코드',
 '제품명',
 '규격',
 '급여구분',
 '약효분류번호',
 '전문일반구분',
 '주성분코드',
 '사용장려비용가산',
 '예외약1',
 '적용개시일자',
 '업체명',
 '단위',
 '투여경로',
 '저가필수구분',
 '생동인정제품',
 '임의조제불가구분',
 '상한가',
 '예외약2',
 '적용종료일자']
len(cols)

19

In [14]:
pd.options.display.max_columns = 25

In [15]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,,642901951,일동후루마린주사0.5그램(플로목세프나트륨)_(0.5g/1병),1,급여,618,전문,159501BIJ,0,-,2025-04-21,,,일동제약(주),병,주사,,,,"6,294",-,,
1,,641805900,광동시타글립틴인산염수화물정100밀리그램_(0.1285g/1정),1,급여,396,전문,501103ATB,0,-,2024-09-02,,,광동제약(주),정,내복,,,,487,-,,
2,,643202640,스타비아정100mg(시타글립틴인산염수화물)_(0.1285g/1정),1,급여,396,전문,501103ATB,0,-,2024-09-02,,,동성제약(주),정,내복,,생동성시험,,487,-,,


In [16]:
df = df.drop([0, 11, 12, 22], axis=1)

In [17]:
df.shape

(10, 19)

In [18]:
len(cols)

19

In [19]:
df.columns = cols

In [20]:
df.head()

,제품코드,제품명,규격,급여구분,약효분류번호,전문일반구분,주성분코드,사용장려비용가산,예외약1,적용개시일자,업체명,단위,투여경로,저가필수구분,생동인정제품,임의조제불가구분,상한가,예외약2,적용종료일자
0,642901951,일동후루마린주사0.5그램(플로목세프나트륨)_(0.5g/1병),1,급여,618,전문,159501BIJ,0,-,2025-04-21,일동제약(주),병,주사,,,,"6,294",-,
1,641805900,광동시타글립틴인산염수화물정100밀리그램_(0.1285g/1정),1,급여,396,전문,501103ATB,0,-,2024-09-02,광동제약(주),정,내복,,,,487,-,
2,643202640,스타비아정100mg(시타글립틴인산염수화물)_(0.1285g/1정),1,급여,396,전문,501103ATB,0,-,2024-09-02,동성제약(주),정,내복,,생동성시험,,487,-,
3,648507280,시타글루정100밀리그램(시타글립틴인산염수화물)_(0.1285g/1정),1,급여,396,전문,501103ATB,0,-,2024-09-02,신풍제약(주),정,내복,,,,487,-,
4,648507360,시타글루정25밀리그램(시타글립틴인산염수화물)_(32.125mg/1정),1,급여,396,전문,501101ATB,0,-,2024-09-02,신풍제약(주),정,내복,,,,215,-,


In [21]:
def get_medicine_info(url, params):
    table = get_medicine_table(url, params)
    try:
        trs = table.select("tbody > tr")
        drug = parse_tr(trs)
        df = pd.DataFrame(drug)
        df = df.drop([0, 11, 12, 22], axis=1)
        cols = ['제품코드',
             '제품명',
             '규격',
             '급여구분',
             '약효분류번호',
             '전문일반구분',
             '주성분코드',
             '사용장려비용가산',
             '예외약1',
             '적용개시일자',
             '업체명',
             '단위',
             '투여경로',
             '저가필수구분',
             '생동인정제품',
             '임의조제불가구분',
             '상한가',
             '예외약2',
             '적용종료일자']

        df.columns = cols
        return df
    except:
        return False

In [22]:
params["artcNm"] = "111001ATE"
get_medicine_info(url, params)

,제품코드,제품명,규격,급여구분,약효분류번호,전문일반구분,주성분코드,사용장려비용가산,예외약1,적용개시일자,업체명,단위,투여경로,저가필수구분,생동인정제품,임의조제불가구분,상한가,예외약2,적용종료일자
0,054000520,코스맥스아스피린장용정100mg(아스피린)_(0.1g/1정),1,삭제,219,일반,111001ATE,0,-,2021-02-01,코스맥스바이오(주),정,내복,,,,0,-,
1,644308680,아센정(아스피린)_(0.1g/1정),1,삭제,219,일반,111001ATE,0,-,2020-11-01,한국유나이티드제약(주),정,내복,,,,0,-,
2,622600430,씨엘팜아스피린장용정100mg(아스피린)_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-08-01,(주)씨엘팜,정,내복,,,,77,-,
3,674401990,아이스피린장용정(아스피린)_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-08-01,아이큐어(주),정,내복,,,,77,-,
4,057001120,에스피린장용정100밀리그램(아스피린)_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-08-01,주식회사 더유제약,정,내복,,,,77,-,
5,642400970,영진아스피린장용정_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-07-01,영진약품(주),정,내복,,생동성시험,,33,-,
6,642308330,삼성아스피린장용정100밀리그램_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-06-01,삼성제약(주),정,내복,,,,77,-,
7,670501970,보령바이오아스피린장용정100밀리그램_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-03-01,(주)보령바이오파마,정,내복,,,,77,-,
8,653805650,신일아스피린장용정100밀리그램_(0.1g/1정),1,급여,219,일반,111001ATE,0,-,2020-03-01,신일제약(주),정,내복,,,,77,-,
9,648507210,신풍아스피린장용정100밀리그램_(0.1g/1정),1,삭제,219,일반,111001ATE,0,-,2020-02-01,신풍제약(주),정,내복,,,,0,-,
